In [ ]:

from iduedu.enums.network_enums import Network

import pandas as pd
import numpy as np
from shapely.geometry import LineString
import networkx as nx
from shapely import line_merge, MultiLineString
import geopandas as gpd
from iduedu import get_4326_boundary

way_filter = Network.DRIVE.filter
simplify = True
local_crs = 32636

from iduedu.modules.overpass_downloaders import get_network_by_filters
from iduedu import config

config.set_logger_lvl('DEBUG')

bounds = get_4326_boundary(osm_id=337422)
data = get_network_by_filters(bounds, way_filter)

# Разделяем на ноды и линии
nodes = data[data['type'] == 'node'].copy()
ways = data[data['type'] == 'way'].copy()

# Собираем все координаты в лист
coords_list = [
    np.asarray([(p["lon"], p["lat"]) for p in pts], dtype="f8")
    for pts in ways["geometry"].values
]

# определяем начала и концы каждого сегмента
starts = np.concatenate([a[:-1] for a in coords_list], axis=0)
ends = np.concatenate([a[1:] for a in coords_list], axis=0)
M = starts.shape[0]

# определяем длины каждого way для ссылок на индекс
lengths = np.array([a.shape[0] for a in coords_list], dtype=int)
seg_counts = np.maximum(lengths - 1, 0)

# сохраняем ссылки на индексы в ways
way_idx = np.repeat(ways.index.values, seg_counts)

geoms = [LineString([tuple(s), tuple(e)]) for s, e in zip(starts, ends)]

edges = gpd.GeoDataFrame(
    {"way_idx": way_idx},
    geometry=geoms,
    crs=4326
).to_crs(local_crs)
edges = edges.join(ways[["id", "tags"]], on="way_idx")

if simplify:
    simplified_lines = list(line_merge(MultiLineString(edges.geometry.to_list()), directed=True).geoms)
    simplified_lines = gpd.GeoDataFrame(geometry=simplified_lines, crs=local_crs)

    # точки середины линий и sjoin на ближайшую
    line_mid_points = simplified_lines.copy()
    line_mid_points.geometry = line_mid_points.interpolate(simplified_lines.length / 2)

    joined = gpd.sjoin_nearest(line_mid_points[["geometry"]], edges, how="left", max_distance=1)

    joined = joined.reset_index().drop_duplicates(subset="index").set_index("index")

    simplified_lines = simplified_lines.join(joined[['tags']])

    edges = simplified_lines

# Раскрываем необходимые теги
TAG_WHITELIST = ["oneway", "highway", "name", "maxspeed", "lanes"]

tags_data = pd.DataFrame.from_records(
    (
        {k: v for k, v in d.items() if k in TAG_WHITELIST}
        if isinstance(d, dict) else {}
        for d in edges["tags"]
    ),
    index=edges.index
)

edges = edges.join(tags_data)

# определить двусторонние дороги и продублировать перевернув

two_way = edges[edges["oneway"] != "yes"].copy()
two_way.geometry = two_way.geometry.reverse()

edges = pd.concat([edges, two_way], ignore_index=True)

# формируем ноды начала и конца каждой эджи
coords = edges.geometry.get_coordinates().to_numpy()
counts = edges.geometry.count_coordinates()
cuts = np.cumsum(counts)

first_idx = np.r_[0, cuts[:-1]]
last_idx = cuts - 1

starts = coords[first_idx]
ends = coords[last_idx]

edges["start"] = list(map(tuple, starts))
edges["end"] = list(map(tuple, ends))

all_endpoints = pd.Index(edges["start"]).append(pd.Index(edges["end"]))
labels, uniques = pd.factorize(all_endpoints)
n = len(edges)
u = labels[:n]
v = labels[n:]
attrs = edges[TAG_WHITELIST].to_dict("records")

G = nx.DiGraph()

G.add_nodes_from((i, {"x": float(x), "y": float(y)}) for i, (x, y) in enumerate(uniques))
G.add_edges_from(zip(u, v, attrs))

In [ ]:
nx.write_gml(G, "my_graph_simplify.gml", stringizer=lambda x: str(x))

In [ ]:
G.edges(data=True)

In [ ]:
from iduedu.enums.network_enums import Network

import pandas as pd
import numpy as np
from shapely.geometry import LineString
import networkx as nx
from shapely import line_merge, MultiLineString
import geopandas as gpd
from iduedu import get_4326_boundary

way_filter = Network.DRIVE.filter
simplify = True
local_crs = 32636

from iduedu.modules.overpass_downloaders import get_network_by_filters
from iduedu import config

config.set_logger_lvl('DEBUG')

bounds = get_4326_boundary(osm_id=1114252)

In [ ]:
from osmnx import graph_from_polygon

graph_osmnx = graph_from_polygon(bounds, network_type='drive')

In [ ]:
from iduedu import get_drive_graph

iduedu_graph = get_drive_graph(territory=bounds)

In [ ]:
len(graph_osmnx.edges)

In [ ]:
len(iduedu_graph.edges)

In [ ]:
from pyproj import Transformer
from osmnx import project_graph

graph_osmnx = project_graph(graph_osmnx, to_crs=32636)
for node, data in graph_osmnx.nodes(data=True):
    if len(data) != 0:
        graph_osmnx.nodes(data=True)[node]['x'] = float(data['x'])
        graph_osmnx.nodes(data=True)[node]['y'] = float(data['y'])

In [ ]:
nx.write_gml(graph_osmnx, "graph_osmnx.gml", stringizer=lambda x: str(x))

In [ ]:
nx.write_gml(iduedu_graph, "graph_iduedu.gml", stringizer=lambda x: str(x))

In [ ]:

node_ids = set(ways["start"]).union(ways["end"])
needed_nodes = nodes.set_index('id').loc[list(node_ids)].copy()

In [ ]:
from shapely import Point
import geopandas as gpd

needed_nodes['geometry'] = list(map(Point, needed_nodes["lon"], needed_nodes["lat"]))
needed_nodes = gpd.GeoDataFrame(needed_nodes, geometry="geometry", crs=4326)
needed_nodes.explore()

In [ ]:
from networkx import DiGraph

new_graph = DiGraph()



In [ ]:
for _, row in ways.iterrows():



In [ ]:
from networkx import DiGraph

DiGraph().add_edge()

In [ ]:
ways

In [ ]:
gpd.GeoDataFrame(ways, geometry='geometry', crs=4326).explore()

In [ ]:
import geopandas as gpd

gpd.GeoDataFrame(ways, geometry='geometry', crs=4326).explore()

In [ ]:
line_lengths

In [ ]:
ways['geometry'].explode().apply(pd.Series).to_numpy()

In [ ]:
ways['geometry'].apply(len)

In [ ]:
ways['geometry'].apply(pd.Series)

In [ ]:
pd.json_normalize(ways['geometry'].apply(pd.Series))

In [ ]:
nodes[~nodes['tags'].isna()]
